In [18]:
import os
import numpy as np
import cosmoprimo
from desilike import setup_logging
from scipy.linalg import block_diag
setup_logging()

# data dir in NERSC

NERSC_dir = '/pscratch/sd/j/john0712'
wigglez_data_dir = os.path.join(NERSC_dir, 'Practices','wigglez')
z_effs = [0.22, 0.41, 0.60, 0.78]

### 1. reading the dataset

a : 0.1<z<0.3, b : 0.3<z<0.5, c : 0.5<z<0.7, d : 0.7<z<0.9
a : only 9-hr
b : all regions

In [14]:
def read_wigglez_files(file_type,session, region='9-hr', data_dir = wigglez_data_dir):
    """a : 0.1<z<0.3, b : 0.3<z<0.5, c : 0.5<z<0.7, d : 0.7<z<0.9"""
    if session not in ['a','b', 'c', 'd']:
        
        raise ValueError("Session must be one of 'a', 'b', 'c', or 'd'")
    elif file_type not in ['cov', 'measurements', 'windows']:
        raise ValueError("File type must be one of 'cov', 'measurements', or 'windows'")
    filepath = os.path.join(data_dir, f'wigglez_jan11{session}_{file_type}.txt')
    with open(filepath, 'r') as f:
        data_lines = []
        prev_line_is_header = False
        for line in f:
            if line.startswith('#') and not prev_line_is_header:
                if 'region,' in line:
                    headers = line.split()
                    region_name = headers[headers.index('region,') - 1]
                    if region_name == region:
                        prev_line_is_header = True
                else:
                    continue
            elif line.startswith('#') and prev_line_is_header:
                if 'k / hMpc^-1' in line:
                    continue
                else:
                    break
            else:
                if prev_line_is_header:
                    line_arr = line.split()
                    line_arr = [float(x) for x in line_arr]
                    data_lines.append(line_arr)
                else:
                    continue
    return np.array(data_lines)

kbin=(0,100)

def get_wigglez_dataset(dataset = 'a'):
    klim = (2,20); 
    if dataset == 'a':
        regions = ['9-hr']
    else:
        regions = ['9-hr', '11-hr', '15-hr', '22-hr', '0-hr', '1-hr', '3-hr']

    cov = [read_wigglez_files('cov',dataset, region=reg)[klim[0]:klim[1],klim[0]:klim[1]] for reg in regions]
    
    k_measured = [read_wigglez_files('measurements',dataset, region=reg)[klim[0]:klim[1],0] for reg in regions]
    power_measured = [read_wigglez_files('measurements',dataset, region=reg)[klim[0]:klim[1],3] for reg in regions]
    windows = [read_wigglez_files('windows',dataset, region=reg)[klim[0]:klim[1], kbin[0]:kbin[1]] for reg in regions]
    return k_measured, power_measured, windows, cov

kbands = np.loadtxt(os.path.join(wigglez_data_dir, 'wigglez_jan11kbands.txt'))[kbin[0]:kbin[1]]

In [22]:
from desilike.theories.galaxy_clustering import KaiserTracerPowerSpectrumMultipoles, DirectPowerSpectrumTemplate
from desilike.observables.galaxy_clustering import TracerPowerSpectrumMultipolesObservable
from desilike.likelihoods import ObservablesGaussianLikelihood
from scipy.linalg import block_diag
from desilike import setup_logging
from desilike.theories import Cosmoprimo
from desilike.emulators import Emulator, TaylorEmulatorEngine

def make_wigglez_likelihood(dataset='a', varied_params=None):
    k_measured, power_measured, windows, cov = get_wigglez_dataset(dataset)
    wigglez_cosmo = cosmoprimo.Cosmology(
        omega_cdm=0.1153,
        omega_b=0.02238,
        h=0.689,
        tau_reio = 0.083,
        n_s = 0.964,
        logA = 3.084, engine='class'
    )
    #wigglez_cosmo = Cosmoprimo(fiducial='DESI', **wigglez_params)
    data_name = ['a', 'b', 'c', 'd']
    k_measured, power_measured, windows, cov = get_wigglez_dataset(dataset)
    template = DirectPowerSpectrumTemplate(fiducial=wigglez_cosmo, z=z_effs[data_name.index(dataset)])
    #for param in list(set(['omega_cdm', 'omega_b', 'h', 'tau_reio', 'n_s', 'logA']) - set(varied_params)):
    #    template.params[param].update(fixed=True)
    theory = KaiserTracerPowerSpectrumMultipoles(template=template, k=kbands)
    observables = []
    cov_tot = block_diag(*cov)
    for i in range(len(k_measured)):
        observable = TracerPowerSpectrumMultipolesObservable(theory=theory, data=power_measured[i], wmatrix=windows[i], ells=(0,), k=k_measured[i], kin=kbands, ellsin=(0,))
        observables.append(observable)
    likelihood = ObservablesGaussianLikelihood(observables=observables, covariance=cov_tot)
    for param in list(set(['omega_cdm', 'omega_b', 'h', 'tau_reio', 'n_s', 'logA', 'b1', 'sn0']) - set(varied_params)):
        likelihood.all_params[param].update(fixed=True)
    
    return likelihood


distance transformation : fiducial $\Lambda \mathrm{CDM}$ cosmology with $\Omega_m=0.3$

### WiggleZ b1 varied separate bins

In [88]:
from desilike.theories.galaxy_clustering import KaiserTracerPowerSpectrumMultipoles, DirectPowerSpectrumTemplate
from desilike.observables.galaxy_clustering import TracerPowerSpectrumMultipolesObservable
from desilike.likelihoods import ObservablesGaussianLikelihood
from scipy.linalg import block_diag
from desilike import setup_logging
from desilike.theories import Cosmoprimo
from desilike.emulators import Emulator, TaylorEmulatorEngine

def make_wigglez_likelihood_emulators(dataset='a', varied_params=None):
    k_measured, power_measured, windows, cov = get_wigglez_dataset(dataset)
    wigglez_cosmo = cosmoprimo.Cosmology(
        omega_cdm=0.1153,
        omega_b=0.02238,
        h=0.689,
        tau_reio = 0.083,
        n_s = 0.964,
        logA = 3.084, engine='class'
    )
    #wigglez_cosmo = Cosmoprimo(fiducial='DESI', **wigglez_params)
    data_name = ['a', 'b', 'c', 'd']
    k_measured, power_measured, windows, cov = get_wigglez_dataset(dataset)
    template = DirectPowerSpectrumTemplate(fiducial=wigglez_cosmo, z=z_effs[data_name.index(dataset)])
    for param in list(set(['omega_cdm', 'omega_b', 'h', 'tau_reio', 'n_s', 'logA']) - set(varied_params)):
        template.params[param].update(fixed=True)
    theory = KaiserTracerPowerSpectrumMultipoles(template=template, k=kbands, ells=(0,))
    theory.params['sn0'].update(fixed=True)

    emulator = Emulator(theory, engine=TaylorEmulatorEngine(order={'*':3}))
    emulator.set_samples()
    emulator.fit()

    observables = []
    cov_tot = block_diag(*cov)
    for i in range(len(k_measured)):
        observable = TracerPowerSpectrumMultipolesObservable(theory=theory, data=power_measured[i], wmatrix=windows[i], ells=(0,), k=k_measured[i], kin=kbands, ellsin=(0,))
        observable.init.update(theory=emulator.to_calculator())
        observables.append(observable)
    likelihood = ObservablesGaussianLikelihood(observables=observables, covariance=cov_tot)
    #for param in list(set(['omega_cdm', 'omega_b', 'h', 'tau_reio', 'n_s', 'logA', 'b1', 'sn0']) - set(varied_params)):
    #    likelihood.all_params[param].update(fixed=True)
    
    return likelihood


In [ ]:
setup_logging()
for dataset in ['a', 'b', 'c', 'd']:
    likelihood = make_wigglez_likelihood(dataset, varied_params=['b1'])
    likelihood.all_params['b1'].update(prior={'limits': [0., 10.], 'latex': 'b_1'})
    
    from desilike.samplers import ZeusSampler


    sampler = ZeusSampler(likelihood, save_fn=f'_tests/chain_wigglez_{dataset}.npy', nwalkers=6)
    sampler.run(check={'max_eigen_gr': 0.01}, max_iterations=1e5)

In [3]:
from desilike.samples import Chain
chain_a = Chain.load('_tests/chain_wigglez_a.npy').remove_burnin(0.4)
chain_b = Chain.load('_tests/chain_wigglez_b.npy').remove_burnin(0.4)
chain_c = Chain.load('_tests/chain_wigglez_c.npy').remove_burnin(0.4)
chain_d = Chain.load('_tests/chain_wigglez_d.npy').remove_burnin(0.4)

[000005.13] [0/1] 12-20 00:44  Chain                     INFO     Loading _tests/chain_wigglez_a.npy.
[000005.14] [0/1] 12-20 00:44  Chain                     INFO     Loading _tests/chain_wigglez_b.npy.
[000005.14] [0/1] 12-20 00:44  Chain                     INFO     Loading _tests/chain_wigglez_c.npy.
[000005.14] [0/1] 12-20 00:44  Chain                     INFO     Loading _tests/chain_wigglez_d.npy.


In [4]:
print(chain_a.to_stats(tablefmt='pretty'))
print(chain_b.to_stats(tablefmt='pretty'))
print(chain_c.to_stats(tablefmt='pretty'))
print(chain_d.to_stats(tablefmt='pretty'))

+----+--------+-------+--------+-------+-----------------+-----------------+
|    | argmax | mean  | median |  std  | quantile:1sigma | interval:1sigma |
+----+--------+-------+--------+-------+-----------------+-----------------+
| b1 | 0.720  | 0.716 | 0.718  | 0.052 |  -0.054/+0.054  |  -0.047/+0.061  |
+----+--------+-------+--------+-------+-----------------+-----------------+
+----+--------+-------+--------+-------+-----------------+-----------------+
|    | argmax | mean  | median |  std  | quantile:1sigma | interval:1sigma |
+----+--------+-------+--------+-------+-----------------+-----------------+
| b1 | 0.906  | 0.906 | 0.906  | 0.014 |  -0.014/+0.013  |  -0.014/+0.013  |
+----+--------+-------+--------+-------+-----------------+-----------------+
+----+--------+-------+--------+-------+-----------------+-----------------+
|    | argmax | mean  | median |  std  | quantile:1sigma | interval:1sigma |
+----+--------+-------+--------+-------+-----------------+-----------------+

In [6]:
chains = [chain_a, chain_b, chain_c, chain_d]
wigglez_cosmo = cosmoprimo.Cosmology(
        omega_cdm=0.1153,
        omega_b=0.02238,
        h=0.689,
        tau_reio = 0.083,
        n_s = 0.964,
        logA = 3.084, engine='class'
    )
data_name = ['a', 'b', 'c', 'd']
for i in range(4):
    growth = wigglez_cosmo.growth_rate(z_effs[i])
    b1 = chains[i].argmax()[0].value
    print(b1)
    b_eff_sq = b1**2 + (2/3)*b1*growth + (1/5)*growth**2
    print(f'b_eff_sq at data {data_name[i]} : {b_eff_sq}')
    

0.7195886032297978
b_eff_sq at data a : 0.895196825165766
0.9064800728867709
b_eff_sq at data b : 1.3497051176100818
1.0645660211160872
b_eff_sq at data c : 1.8014663276655798
1.1542098516210733
b_eff_sq at data d : 2.0981789000815145


In [15]:
from desilike.samples import plotting
from getdist import plots
g = plots.get_subplot_plotter(width_inch=5)
plotting.plot_triangle(chains, labels=['z0.1-0.3', 'z0.3-0.5', 'z0.5-0.7', 'z0.7-0.9'],g=g, fn='Practices/figs/WiggleZ_b1_separate.pdf')

Removed no burn in
Removed no burn in
Removed no burn in
Removed no burn in
[000006.12] [0/1] 12-18 22:24  Plotting                  INFO     Saving figure to Practices/figs/WiggleZ_b1_separate.pdf.


### WiggleZ run second chains with omega_cdm, b1 varied, individual redshift bins

In [7]:
setup_logging()
for dataset in ['a', 'b', 'c', 'd']:
    likelihood = make_wigglez_likelihood(dataset, varied_params=['omega_cdm', 'b1'])
    likelihood.all_params['b1'].update(prior={'limits': [0., 5.], 'latex': 'b_1'})
    likelihood.all_params['omega_cdm'].update(prior={'limits': [0.001, 0.4], 'latex': r'\omega_{cdm}'})
    
    #likelihood.all_params['h'].update(prior={'limits': [0.4, 1.0], 'latex': 'h'})
    
    from desilike.samplers import MCMCSampler
    sampler = MCMCSampler(likelihood, save_fn=f'_tests/chain_wigglez_{dataset}_b1_omega_cdm.npy')
    sampler.run(check={'max_eigen_gr': 0.01}, max_iterations=15000)

[000001.04] [0/1] 12-20 02:38  MCMCSampler               INFO     Varied parameters: ['omega_cdm', 'b1'].
[000011.02] [0/1] 12-20 02:38  BasePipeline              INFO     Found speeds:
[000011.02] [0/1] 12-20 02:38  BasePipeline              INFO     - <desilike.theories.primordial_cosmology.Cosmoprimo object at 0x7f978c66ba90>: 166.96 iterations / second - 0.006 s / iteration
[000011.02] [0/1] 12-20 02:38  BasePipeline              INFO     - <desilike.theories.galaxy_clustering.base.APEffect object at 0x7f978c65cee0>: 23.42 iterations / second - 0.043 s / iteration
[000011.03] [0/1] 12-20 02:38  BasePipeline              INFO     - <desilike.theories.galaxy_clustering.power_template.DirectPowerSpectrumTemplate object at 0x7f978c63a470>: 1.17 iterations / second - 0.855 s / iteration
[000011.03] [0/1] 12-20 02:38  BasePipeline              INFO     - <desilike.theories.galaxy_clustering.full_shape.KaiserPowerSpectrumMultipoles object at 0x7f978c65fd60>: 707.73 iterations / second - 0

/pscratch/sd/j/john0712/install/desilike/desilike/samplers/base.py:177: UserWarning: Error "CosmologyComputationError()" raised is caught up with -inf loglikelihood. Set logging level to debug (setup_logging("debug")) to get full stack trace.
  warnings.warn('Error "{}" raised is caught up with -inf loglikelihood. Set logging level to debug (setup_logging("debug")) to get full stack trace.'.format(repr(error[0])))


[006309.47] [0/1] 12-20 04:23  Chain                     INFO     Saving _tests/chain_wigglez_b_b1_omega_cdm.npy.
[006309.48] [0/1] 12-20 04:23  MCMCSampler               INFO     Diagnostics:
[006309.49] [0/1] 12-20 04:23  Diagnostics               INFO     - max eigen Gelman-Rubin - 1 is 0.0333; not < 0.01.
[006309.50] [0/1] 12-20 04:23  Diagnostics               INFO     - max diag Gelman-Rubin - 1 is 0.00828.
[006309.55] [0/1] 12-20 04:23  Diagnostics               INFO     - max diag Gelman-Rubin - 1 at 1.0 sigmas is 0.0722.
[006309.56] [0/1] 12-20 04:23  Diagnostics               INFO     - max Geweke is 0.301.
[006309.56] [0/1] 12-20 04:23  Diagnostics               INFO     - Geweke p-value is 0.13.
[006309.56] [0/1] 12-20 04:23  Diagnostics               INFO     - effective sample size = (6300 iterations / integrated autocorrelation time) (reliable) is 290.
[006309.56] [0/1] 12-20 04:23  Diagnostics               INFO     - max variation of integrated autocorrelation time is 

In [2]:
from desilike.samples import Chain
chain_a = Chain.load('_tests/chain_wigglez_a_b1_omega_cdm_copy.npy').remove_burnin(0.5)
chain_b = Chain.load('_tests/chain_wigglez_b_b1_omega_cdm_copy.npy').remove_burnin(0.5)
chain_c = Chain.load('_tests/chain_wigglez_c_b1_omega_cdm_copy.npy').remove_burnin(0.5)
chain_d = Chain.load('_tests/chain_wigglez_d_b1_omega_cdm_copy.npy').remove_burnin(0.5)
print(chain_a.to_stats(tablefmt='pretty'))
print(chain_b.to_stats(tablefmt='pretty'))
print(chain_c.to_stats(tablefmt='pretty'))
print(chain_d.to_stats(tablefmt='pretty'))

[000105.36] [0/1] 12-20 20:14  Chain                     INFO     Loading _tests/chain_wigglez_a_b1_omega_cdm_copy.npy.
[000105.47] [0/1] 12-20 20:14  Chain                     INFO     Loading _tests/chain_wigglez_b_b1_omega_cdm_copy.npy.
[000105.47] [0/1] 12-20 20:14  Chain                     INFO     Loading _tests/chain_wigglez_c_b1_omega_cdm_copy.npy.
[000105.48] [0/1] 12-20 20:14  Chain                     INFO     Loading _tests/chain_wigglez_d_b1_omega_cdm_copy.npy.
+-----------+--------+-------+--------+-------+-----------------+-----------------+
|           | argmax | mean  | median |  std  | quantile:1sigma | interval:1sigma |
+-----------+--------+-------+--------+-------+-----------------+-----------------+
| omega_cdm | 0.142  | 0.168 | 0.161  | 0.056 |  -0.063/+0.058  |  -0.071/+0.042  |
|    b1     |  0.59  | 0.55  |  0.52  | 0.20  |   -0.20/+0.22   |   -0.25/+0.12   |
+-----------+--------+-------+--------+-------+-----------------+-----------------+
+-----------+---

In [3]:
chains = [chain_a, chain_b, chain_c, chain_d]
wigglez_cosmo = cosmoprimo.Cosmology(
        omega_cdm=0.1153,
        omega_b=0.02238,
        h=0.689,
        tau_reio = 0.083,
        n_s = 0.964,
        logA = 3.084, engine='class'
    )
data_name = ['a', 'b', 'c', 'd']
for i in range(4):
    omega_cdm = chains[i].argmax()[0].value
    cosmo = wigglez_cosmo.clone(omega_cdm=omega_cdm)
    growth = cosmo.growth_rate(z_effs[i])
    b1 = chains[i].argmax()[1].value
    b_eff_sq = b1**2 + (2/3)*b1*growth + (1/5)*growth**2
    print(f'b_eff_sq at data {data_name[i]} : {b_eff_sq}')

b_eff_sq at data a : 0.7084834811050869
b_eff_sq at data b : 1.254100136601398
b_eff_sq at data c : 1.6136758534790578
b_eff_sq at data d : 2.412258549139418


In [4]:
from desilike.samples import plotting
from getdist import plots
g = plots.get_subplot_plotter(width_inch=5)
plotting.plot_triangle([chain_a, chain_b, chain_c, chain_d], g=g,labels=['0.1<z<0.3', '0.3<z<0.5', '0.5<z<0.7', '0.7<z<0.9'], fn='Practices/figs/WiggleZ_separate_bins.pdf')

Removed no burn in
Removed no burn in
Removed no burn in
Removed no burn in
[000121.12] [0/1] 12-20 20:14  Plotting                  INFO     Saving figure to Practices/figs/WiggleZ_separate_bins.pdf.


### WiggleZ merge multiple redshift observations

In [7]:
from desilike.theories.galaxy_clustering import KaiserTracerPowerSpectrumMultipoles, DirectPowerSpectrumTemplate
from desilike.observables.galaxy_clustering import TracerPowerSpectrumMultipolesObservable
from desilike.likelihoods import ObservablesGaussianLikelihood, SumLikelihood
from scipy.linalg import block_diag
from desilike.theories import Cosmoprimo
from desilike.emulators import Emulator, TaylorEmulatorEngine

def make_wigglez_total_likelihood(varied_params=None):
    
    wigglez_cosmo = cosmoprimo.Cosmology(
        omega_cdm=0.1153,
        omega_b=0.02238,
        h=0.689,
        tau_reio = 0.083,
        n_s = 0.964,
        logA = 3.084, engine='class'
    )
    #wigglez_cosmo = Cosmoprimo(fiducial='DESI', **wigglez_params)
    data_name = ['a', 'b', 'c', 'd']
    likelihoods = []
    for i in data_name:
        k_measured, power_measured, windows, cov = get_wigglez_dataset(i)
        template = DirectPowerSpectrumTemplate(fiducial=wigglez_cosmo, z=z_effs[data_name.index(i)])
        #for param in list(set(['omega_cdm', 'omega_b', 'h', 'tau_reio', 'n_s', 'logA']) - set(varied_params)):
        #    template.params[param].update(fixed=True)
        theory = KaiserTracerPowerSpectrumMultipoles(template=template, k=kbands)
        theory.params['b1'].update(namespace=f'reg_{i}')
        observables = []
        cov_tot = block_diag(*cov)
        for i in range(len(k_measured)):
            observable = TracerPowerSpectrumMultipolesObservable(theory=theory, data=power_measured[i], wmatrix=windows[i], ells=(0,), k=k_measured[i], kin=kbands, ellsin=(0,))
            observables.append(observable)
        likelihood = ObservablesGaussianLikelihood(observables=observables, covariance=cov_tot)
        likelihoods.append(likelihood)
    total_likelihood = SumLikelihood(likelihoods=likelihoods)
    for param in list(set(['omega_cdm', 'omega_b', 'h', 'tau_reio', 'n_s', 'logA', 'sn0']) - set(varied_params)):
        total_likelihood.all_params[param].update(fixed=True)
    
    return total_likelihood

In [8]:
from desilike.samplers import MCMCSampler
total_likelihood = make_wigglez_total_likelihood(varied_params=['omega_cdm'])
sampler = MCMCSampler(total_likelihood, save_fn=f'../_tests/chain_wigglez_tot_b1_omega_cdm.npy')
sampler.run(check={'max_eigen_gr': 0.05}, max_iterations=120000)

/pscratch/sd/j/john0712/install/desilike/desilike/base.py:447: UserWarning: Derived parameter loglikelihood of <desilike.likelihoods.base.ObservablesGaussianLikelihood object at 0x7fd9684df970> is already derived in <desilike.likelihoods.base.ObservablesGaussianLikelihood object at 0x7fd9684fa230>.
  if self.mpicomm.rank == 0: warnings.warn(msg)
/pscratch/sd/j/john0712/install/desilike/desilike/base.py:447: UserWarning: Derived parameter logprior of <desilike.likelihoods.base.ObservablesGaussianLikelihood object at 0x7fd9684df970> is already derived in <desilike.likelihoods.base.ObservablesGaussianLikelihood object at 0x7fd9684fa230>.
  if self.mpicomm.rank == 0: warnings.warn(msg)
/pscratch/sd/j/john0712/install/desilike/desilike/base.py:447: UserWarning: Derived parameter loglikelihood of <desilike.likelihoods.base.ObservablesGaussianLikelihood object at 0x7fd9684fb760> is already derived in <desilike.likelihoods.base.ObservablesGaussianLikelihood object at 0x7fd9684df970>.
  if self

[000019.33] [0/1] 12-13 05:18  MCMCSampler               INFO     Varied parameters: ['omega_cdm', 'reg_d.b1', 'reg_c.b1', 'reg_b.b1', 'reg_a.b1'].
[000061.28] [0/1] 12-13 05:19  BasePipeline              INFO     Found speeds:
[000061.29] [0/1] 12-13 05:19  BasePipeline              INFO     - <desilike.theories.primordial_cosmology.Cosmoprimo object at 0x7fd9683c6f80>: 165.08 iterations / second - 0.006 s / iteration
[000061.29] [0/1] 12-13 05:19  BasePipeline              INFO     - <desilike.theories.galaxy_clustering.base.APEffect object at 0x7fd9683c7df0>: 26.83 iterations / second - 0.037 s / iteration
[000061.29] [0/1] 12-13 05:19  BasePipeline              INFO     - <desilike.theories.galaxy_clustering.power_template.DirectPowerSpectrumTemplate object at 0x7fd96830e3e0>: 1.15 iterations / second - 0.869 s / iteration
[000061.29] [0/1] 12-13 05:19  BasePipeline              INFO     - <desilike.theories.galaxy_clustering.full_shape.KaiserPowerSpectrumMultipoles object at 0x7fd

/pscratch/sd/j/john0712/install/desilike/desilike/samples/diagnostics.py:339: RuntimeWarning: divide by zero encountered in scalar divide
  diff /= (np.cov(value_first, aweights=aweight_first, fweights=fweight_first) + np.cov(value_last, aweights=aweight_last, fweights=fweight_last))**0.5
/global/common/software/desi/users/adematti/perlmutter/cosmodesiconda/20240118-1.0.0/conda/lib/python3.10/site-packages/scipy/stats/_stats_py.py:1070: RuntimeWarning: invalid value encountered in subtract
  a_zero_mean = a - mean


[000576.90] [0/1] 12-13 05:28  MCMCSampler               INFO     352 steps, acceptance rate 0.511.
[000601.98] [0/1] 12-13 05:28  Chain                     INFO     Saving ../_tests/chain_wigglez_tot_b1_omega_cdm.npy.
[000601.99] [0/1] 12-13 05:28  MCMCSampler               INFO     Diagnostics:
[000602.00] [0/1] 12-13 05:28  Diagnostics               INFO     - max eigen Gelman-Rubin - 1 is 2.96; not < 0.05.
[000602.02] [0/1] 12-13 05:28  Diagnostics               INFO     - max diag Gelman-Rubin - 1 is 2.7.
[000602.08] [0/1] 12-13 05:28  Diagnostics               INFO     - max diag Gelman-Rubin - 1 at 1.0 sigmas is 7.19.
[000602.09] [0/1] 12-13 05:28  Diagnostics               INFO     - max Geweke is 2.76e+13.
[000602.10] [0/1] 12-13 05:28  Diagnostics               INFO     - Geweke p-value is 6.05e-12.
[000602.10] [0/1] 12-13 05:28  Diagnostics               INFO     - effective sample size = (450 iterations / integrated autocorrelation time) is 10.8.
[000602.10] [0/1] 12-13 05:

[Chain(shape=(25200,), params=ParameterCollection(['omega_cdm', 'reg_c.b1', 'reg_d.b1', 'reg_b.b1', 'reg_a.b1', 'fweight', 'logposterior', 'h', 'omega_b', 'logA', 'n_s', 'tau_reio', 'm_ncdm', 'N_eff', 'w0_fld', 'wa_fld', 'Omega_k', 'sigmapar', 'sigmaper', 'sn0', 'loglikelihood', 'logprior']))]

In [19]:
from desilike.samples import Chain, plotting

chain_tot = Chain.load('_tests/chain_wigglez_tot_b1_omega_cdm.npy').remove_burnin(0.5)
g = plots.get_subplot_plotter(width_inch=12)
print(chain_tot.to_stats(tablefmt='pretty'))
plotting.plot_triangle(chain_tot, params=['omega_cdm', 'reg_a.b1', 'reg_b.b1', 'reg_c.b1', 'reg_d.b1'], g=g,fn='Practices/figs/WiggleZ_merged.pdf', settings={'smooth_scale_2D' : 2.})

[001658.03] [0/1] 12-18 22:52  Chain                     INFO     Loading _tests/chain_wigglez_tot_b1_omega_cdm.npy.
+-----------+--------+--------+--------+--------+-----------------+-----------------+
|           | argmax |  mean  | median |  std   | quantile:1sigma | interval:1sigma |
+-----------+--------+--------+--------+--------+-----------------+-----------------+
| omega_cdm | 0.1181 | 0.1193 | 0.1190 | 0.0057 | -0.0054/+0.0056 | -0.0055/+0.0054 |
| reg_c.b1  | 1.043  | 1.036  | 1.036  | 0.043  |  -0.044/+0.040  |  -0.047/+0.034  |
| reg_d.b1  | 1.132  | 1.126  | 1.126  | 0.046  |  -0.045/+0.044  |  -0.043/+0.045  |
| reg_b.b1  | 0.889  | 0.882  | 0.883  | 0.038  |  -0.040/+0.036  |  -0.037/+0.039  |
| reg_a.b1  | 0.699  | 0.694  | 0.692  | 0.057  |  -0.056/+0.056  |  -0.056/+0.057  |
+-----------+--------+--------+--------+--------+-----------------+-----------------+
Removed no burn in
[001660.49] [0/1] 12-18 22:52  Plotting                  INFO     Saving figure to Practic

### Wigglez method

In [24]:
def scaling_factor(**kwargs):
    # requires 'z_eff' and 'template' in kwargs 'template' is of type KaiserTracerPowerSpectrumMultipoles
    kaiser_template = kwargs['template']
    template_ba = kaiser_template.fiducial.get_background(engine='class')
    z_eff = kaiser_template.z
    
    fid_cosmo = cosmoprimo.Cosmology(Omega_m=0.3, h=0.71, Omega_b=0.166*0.3, sigma8=0.8, n_s=0.96)
    ba_fid = fid_cosmo.get_background(engine='class')
    D_A_zeff, H_zeff= template_ba.angular_diameter_distance(z_eff), template_ba.hubble_function(z_eff)
    D_A_zeff_fid = ba_fid.angular_diameter_distance(z_eff)
    H_zeff_fid = ba_fid.hubble_function(z_eff)

    a_scale = (D_A_zeff**2*H_zeff/D_A_zeff_fid**2/H_zeff_fid)**(1/3)
    return a_scale

In [81]:
def calculate_b_eff(dataset = 'a'):
    _cache = ['a', 'b', 'c', 'd']
    k_measured, power_measured, windows, cov = get_wigglez_dataset(dataset=dataset)

    wigglez_cosmo = cosmoprimo.Cosmology(
        omega_cdm=0.1153,
        omega_b=0.02238,
        H0=68.9,
        tau_reio = 0.083,
        n_s = 0.964,
        logA = 3.084, engine='class'
    )
    tot_pk_convolved = []
    tot_pk_obs = np.concatenate(power_measured)
    cov_merged = block_diag(*cov)
    inv_cov_merged = np.linalg.inv(cov_merged)
    template = DirectPowerSpectrumTemplate(z=z_effs[_cache.index(dataset)],fiducial=wigglez_cosmo, k = kbands) # ap effect?
    a_scl = scaling_factor(template=template)
    template.init.update(k = kbands / a_scl)
    template()
    pk_th = template.pk_dd
    for window_i in zip(windows):
        pk_convolved = ((window_i @ pk_th.T) / a_scl**3)[0]
        tot_pk_convolved.append(pk_convolved)

    tot_pk_convolved = np.concatenate(tot_pk_convolved)
    b_eff_squared = tot_pk_convolved @ inv_cov_merged @ tot_pk_obs.T / (tot_pk_convolved @ inv_cov_merged @ tot_pk_convolved.T)
    return b_eff_squared

def plot_b_eff(dataset='a', params=[0.123,0.858]):
    _cache = ['a', 'b', 'c', 'd']
    _regions = ['9-hr', '11-hr', '15-hr', '22-hr', '0-hr', '1-hr', '3-hr']
    k_measured, power_measured, windows, cov = get_wigglez_dataset(dataset=dataset)
    wigglez_cosmo = cosmoprimo.Cosmology(
        omega_cdm=0.1153,
        omega_b=0.02238,
        H0=68.9,
        tau_reio = 0.083,
        n_s = 0.964,
        logA = 3.084, engine='class'
    )
    import matplotlib.pyplot as plt
    plt.close('all')
    b_eff_sq = calculate_b_eff(dataset=dataset)
    template = DirectPowerSpectrumTemplate(z=z_effs[_cache.index(dataset)],fiducial=wigglez_cosmo, k = kbands) # ap effect?
    template2 = DirectPowerSpectrumTemplate(z=z_effs[_cache.index(dataset)],fiducial=wigglez_cosmo, k = kbands) # ap effect?
    theory = KaiserTracerPowerSpectrumMultipoles(template=template2, k=kbands)
    a_scl = scaling_factor(template=template)
    template.init.update(k = kbands / a_scl)
    template()
    pk_th = template.pk_dd
    for i in range(len(windows)):
        pk_convolved = ((windows[i] @ pk_th.T) / a_scl**3)
        #print(pk_convolved)
        observable = TracerPowerSpectrumMultipolesObservable(theory=theory, data=power_measured[i], wmatrix=windows[i], ells=(0,), k=k_measured[i], kin=kbands, ellsin=(0,))
        observable(omega_cdm=params[0], b1=params[1])
        observable = observable.theory
        #print(observable[0])
        plt.figure(figsize=(8,6))
        plt.errorbar(k_measured[i], power_measured[i], yerr=np.sqrt(np.diag(cov[i])), fmt='o', label=f'{_regions[i]} Region')
        plt.plot(k_measured[i], pk_convolved * b_eff_sq, label=f'Analytically marginalized')
        plt.plot(k_measured[i], observable[0], label=f'theory omega_cdm={params[0]}, b1={params[1]}', linestyle='--')
        plt.ylabel(r'$P(k)$ [$h^{-3}$ Mpc$^3$]')
        plt.xlabel(r'$k$ [$h$ Mpc$^{-1}$]')
        plt.legend()
        plt.savefig(f'Practices/figs/WiggleZ_{dataset}_{_regions[i]}_b_eff_fit.pdf')

In [25]:
wigglez_dataset = ['a', 'b', 'c', 'd']
for i in range(4):
    growth = wigglez_cosmo.growth_rate(z_effs[i])
    b_eff_sq = calculate_b_eff(dataset=wigglez_dataset[i])
    print(r'b_{eff}^2',f'for dataset {wigglez_dataset[i]}: {b_eff_sq:.3f}')
    _a = 2/3 * growth
    _b = 1/5 * growth**2 - b_eff_sq
    b1_analytic = 1/2 * ((_a**2 - 4 * _b)**0.5 - _a)
    print('b1 : ', f'{b1_analytic:.3f}')

b_{eff}^2 for dataset a: 0.882
b1 :  0.712
b_{eff}^2 for dataset b: 1.330
b1 :  0.898
b_{eff}^2 for dataset c: 1.775
b1 :  1.055
b_{eff}^2 for dataset d: 2.067
b1 :  1.143


In [74]:
wigglez_cosmo = cosmoprimo.Cosmology(
    omega_cdm=0.1153,
    omega_b=0.02238,
    H0=68.9,
    tau_reio = 0.083,
    n_s = 0.964,
    logA = 3.084, engine='class'
)
template = DirectPowerSpectrumTemplate(z=z_effs[1],fiducial=wigglez_cosmo, k = kbands)
theory = KaiserTracerPowerSpectrumMultipoles(template=template, k=kbands)
k_measured, power_measured, windows, cov = get_wigglez_dataset(dataset='b')
observable = TracerPowerSpectrumMultipolesObservable(theory=theory, data=power_measured[1], wmatrix=windows[1], ells=(0,), k=k_measured[1], kin=kbands, ellsin=(0,))
#observable.varied_params.update(omega_cdm=0.123, b1=0.858)
observable(omega_cdm=0.123, b1=0.858)

In [82]:
plot_b_eff(dataset='b', params=[0.123,0.858])

# DESI FS data

### 1. b1 only separate LRG redshift bins

In [ ]:
from desilike.theories.galaxy_clustering import KaiserTracerPowerSpectrumMultipoles, DirectPowerSpectrumTemplate
from desilike.observables.galaxy_clustering import TracerPowerSpectrumMultipolesObservable
from desilike.observables import ObservableCovariance
from desilike.likelihoods import ObservablesGaussianLikelihood
from scipy.linalg import block_diag
forfit_dir = Path('/global/cfs/cdirs/desi//survey/catalogs/Y1/LSS/iron/LSScats/v1.5/unblinded/desipipe/forfit_2pt/')

wigglez_cosmo = cosmoprimo.Cosmology(
        omega_cdm=0.1153,
        omega_b=0.02238,
        H0=68.9,
        tau_reio = 0.083,
        n_s = 0.964,
        logA = 3.084, engine='class'
    )
zrange = [(0.4, 0.6), (0.6, 0.8), (0.8, 1.1)]
for z in zrange:
    # call in the data
    ref_fn = forfit_dir / f'forfit_power_syst-rotation-hod-photo_klim_0-0.02-0.20_2-0.02-0.20_4-0.02-0.20_LRG_GCcomb_z{z[0]}-{z[1]}.npy'
    covariance = ObservableCovariance.load(ref_fn).select(projs=0, select_projs=True)
    cov_mat = covariance.view()
    # P(k)
    observable_power = covariance.observables('power')
    data = observable_power.view()
    zeff = observable_power.attrs['zeff']
    kin = observable_power.attrs['kin']
    wmatrix = observable_power.attrs['wmatrix'][:36,:kin.shape[0]]
    kout = np.arange(0.02,0.2,0.005)
    wshotnoise = observable_power.attrs['wshotnoise'][:36]

    # create the likelihood
    print(f'Creating likelihood for z range {z[0]}-{z[1]} with zeff={zeff}')
    template = DirectPowerSpectrumTemplate(fiducial=wigglez_cosmo, z=zeff)
    theory = KaiserTracerPowerSpectrumMultipoles(template=template, k=kin)
    observable = TracerPowerSpectrumMultipolesObservable(theory=theory, data=data, wmatrix=wmatrix,ells=(0,), ellsin=(0,),k=kout,kin=kin, wshotnoise=wshotnoise)
    likelihood = ObservablesGaussianLikelihood(observables=observable, covariance=cov_mat)
    for param in likelihood.varied_params:
        if param.name != 'b1':
            likelihood.varied_params[param.name].update(fixed=True)
    likelihood.all_params['b1'].update(prior={'limits': [0., 10.], 'latex': 'b_1'})
    from desilike.samplers import ZeusSampler


    sampler = ZeusSampler(likelihood, save_fn=f'_tests/chain_desi_lrg_{z[0]}-{z[1]}.npy', nwalkers=6)
    sampler.run(check={'max_eigen_gr': 0.01}, max_iterations=1e5)
    
    

[000247.92] [0/1] 12-08 05:09  ObservableCovariance      INFO     Loading /global/cfs/cdirs/desi/survey/catalogs/Y1/LSS/iron/LSScats/v1.5/unblinded/desipipe/forfit_2pt/forfit_power_syst-rotation-hod-photo_klim_0-0.02-0.20_2-0.02-0.20_4-0.02-0.20_LRG_GCcomb_z0.4-0.6.npy.
Creating likelihood for z range 0.4-0.6 with zeff=0.5096288678782911
[000251.14] [0/1] 12-08 05:09  ZeusSampler               INFO     Varied parameters: ['b1'].
[000253.91] [0/1] 12-08 05:09  ZeusSampler               INFO     Successfully vmap input likelihood.
[000254.05] [0/1] 12-08 05:09  ZeusSampler               INFO     Successfully jit input likelihood.
[000254.20] [0/1] 12-08 05:09  root                      INFO     Initialising ensemble of 6 walkers...
[000265.15] [0/1] 12-08 05:09  Chain                     INFO     Saving _tests/chain_desi_lrg_0.4-0.6.npy.
[000265.16] [0/1] 12-08 05:09  ZeusSampler               INFO     Diagnostics:
[000265.17] [0/1] 12-08 05:09  Diagnostics               INFO     - max e

In [20]:
chain_04_06 = Chain.load('_tests/chain_desi_lrg_0.4-0.6.npy').remove_burnin(0.4)
chain_06_08 = Chain.load('_tests/chain_desi_lrg_0.6-0.8.npy').remove_burnin(0.4)
chain_08_11 = Chain.load('_tests/chain_desi_lrg_0.8-1.1.npy').remove_burnin(0.4)
print(chain_04_06.to_stats(tablefmt='pretty'))
print(chain_06_08.to_stats(tablefmt='pretty'))
print(chain_08_11.to_stats(tablefmt='pretty'))

[001706.69] [0/1] 12-18 22:53  Chain                     INFO     Loading _tests/chain_desi_lrg_0.4-0.6.npy.
[001706.70] [0/1] 12-18 22:53  Chain                     INFO     Loading _tests/chain_desi_lrg_0.6-0.8.npy.
[001706.71] [0/1] 12-18 22:53  Chain                     INFO     Loading _tests/chain_desi_lrg_0.8-1.1.npy.
+----+--------+-------+--------+-------+-----------------+-----------------+
|    | argmax | mean  | median |  std  | quantile:1sigma | interval:1sigma |
+----+--------+-------+--------+-------+-----------------+-----------------+
| b1 | 1.717  | 1.717 | 1.717  | 0.017 |  -0.017/+0.017  |  -0.017/+0.016  |
+----+--------+-------+--------+-------+-----------------+-----------------+
+----+--------+-------+--------+-------+-----------------+-----------------+
|    | argmax | mean  | median |  std  | quantile:1sigma | interval:1sigma |
+----+--------+-------+--------+-------+-----------------+-----------------+
| b1 | 1.876  | 1.876 | 1.876  | 0.017 |  -0.017/+0.017  

In [22]:
g=plots.get_subplot_plotter(width_inch=5)
plotting.plot_triangle([chain_04_06, chain_06_08, chain_08_11],labels=['z04-06', 'z06-08', 'z08-11'], g=g,fn='Practices/figs/Desi_LRG_b1_separate_bins.pdf')

Removed no burn in
Removed no burn in
Removed no burn in
[001714.34] [0/1] 12-18 22:53  Plotting                  INFO     Saving figure to Practices/figs/Desi_LRG_b1_separate_bins.pdf.


### DESI LRG FS with omega_m, b1

In [11]:
from desilike.theories.galaxy_clustering import KaiserTracerPowerSpectrumMultipoles, DirectPowerSpectrumTemplate
from desilike.observables.galaxy_clustering import TracerPowerSpectrumMultipolesObservable
from desilike.likelihoods import ObservablesGaussianLikelihood
from desilike.observables import ObservableCovariance
from pathlib import Path
from desilike import setup_logging
forfit_dir = Path('/global/cfs/cdirs/desi//survey/catalogs/Y1/LSS/iron/LSScats/v1.5/unblinded/desipipe/forfit_2pt/')
wigglez_cosmo = cosmoprimo.Cosmology(
        omega_cdm=0.1153,
        omega_b=0.02238,
        H0=68.9,
        tau_reio = 0.083,
        n_s = 0.964,
        logA = 3.084, engine='class'
    )
zrange = [(0.4, 0.6), (0.6, 0.8), (0.8, 1.1)]
for z in zrange:
    # call in the data
    ref_fn = forfit_dir / f'forfit_power_syst-rotation-hod-photo_klim_0-0.02-0.20_2-0.02-0.20_4-0.02-0.20_LRG_GCcomb_z{z[0]}-{z[1]}.npy'
    covariance = ObservableCovariance.load(ref_fn).select(projs=0, select_projs=True)
    cov_mat = covariance.view()
    # P(k)
    observable_power = covariance.observables('power')
    data = observable_power.view()
    zeff = observable_power.attrs['zeff']
    kin = observable_power.attrs['kin']
    wmatrix = observable_power.attrs['wmatrix'][:36,:kin.shape[0]]
    kout = np.arange(0.02,0.2,0.005)
    wshotnoise = observable_power.attrs['wshotnoise'][:36]

    # create the likelihood
    print(f'Creating likelihood for z range {z[0]}-{z[1]} with zeff={zeff}')
    template = DirectPowerSpectrumTemplate(fiducial=wigglez_cosmo, z=zeff)
    theory = KaiserTracerPowerSpectrumMultipoles(template=template, k=kin)
    observable = TracerPowerSpectrumMultipolesObservable(theory=theory, data=data, wmatrix=wmatrix,ells=(0,), ellsin=(0,),k=kout,kin=kin, wshotnoise=wshotnoise)
    likelihood = ObservablesGaussianLikelihood(observables=observable, covariance=cov_mat)
    for param in likelihood.varied_params:
        if param.name != 'b1' and param.name != 'omega_cdm':
            likelihood.varied_params[param.name].update(fixed=True)
    likelihood.all_params['b1'].update(prior={'limits': [0., 10.], 'latex': 'b_1'})
    from desilike.samplers import MCMCSampler


    sampler = MCMCSampler(likelihood, save_fn=f'../_tests/chain_desi_lrg_b1_omega_cdm_{z[0]}-{z[1]}.npy')
    sampler.run(check={'max_eigen_gr': 0.01}, max_iterations=10000)

[005519.11] [0/1] 12-13 06:50  ObservableCovariance      INFO     Loading /global/cfs/cdirs/desi/survey/catalogs/Y1/LSS/iron/LSScats/v1.5/unblinded/desipipe/forfit_2pt/forfit_power_syst-rotation-hod-photo_klim_0-0.02-0.20_2-0.02-0.20_4-0.02-0.20_LRG_GCcomb_z0.4-0.6.npy.
Creating likelihood for z range 0.4-0.6 with zeff=0.5096288678782911
[005520.22] [0/1] 12-13 06:50  MCMCSampler               INFO     Varied parameters: ['omega_cdm', 'b1'].
[005531.07] [0/1] 12-13 06:50  BasePipeline              INFO     Found speeds:
[005531.07] [0/1] 12-13 06:50  BasePipeline              INFO     - <desilike.theories.primordial_cosmology.Cosmoprimo object at 0x7fd92829d570>: 165.61 iterations / second - 0.006 s / iteration
[005531.07] [0/1] 12-13 06:50  BasePipeline              INFO     - <desilike.theories.galaxy_clustering.base.APEffect object at 0x7fd9680fd180>: 22.56 iterations / second - 0.044 s / iteration
[005531.07] [0/1] 12-13 06:50  BasePipeline              INFO     - <desilike.theorie

/global/common/software/desi/users/adematti/perlmutter/cosmodesiconda/20240118-1.0.0/conda/lib/python3.10/site-packages/scipy/stats/_stats_py.py:1806: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=8
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


[005846.61] [0/1] 12-13 06:56  MCMCSampler               INFO     Updating proposal covariance.
[005877.84] [0/1] 12-13 06:56  MCMCSampler               INFO     466 steps, acceptance rate 0.496.
[005885.95] [0/1] 12-13 06:56  Chain                     INFO     Saving ../_tests/chain_desi_lrg_b1_omega_cdm_0.4-0.6.npy.
[005885.95] [0/1] 12-13 06:56  MCMCSampler               INFO     Diagnostics:
[005885.97] [0/1] 12-13 06:56  Diagnostics               INFO     - max eigen Gelman-Rubin - 1 is 0.179; not < 0.01.
[005885.98] [0/1] 12-13 06:56  Diagnostics               INFO     - max diag Gelman-Rubin - 1 is 0.137.
[005886.01] [0/1] 12-13 06:56  Diagnostics               INFO     - max diag Gelman-Rubin - 1 at 1.0 sigmas is 0.79.
[005886.02] [0/1] 12-13 06:56  Diagnostics               INFO     - max Geweke is 4.64e+14.
[005886.02] [0/1] 12-13 06:56  Diagnostics               INFO     - Geweke p-value is 1.03e-05.
[005886.02] [0/1] 12-13 06:56  Diagnostics               INFO     - effecti

/pscratch/sd/j/john0712/install/desilike/desilike/samples/diagnostics.py:339: RuntimeWarning: divide by zero encountered in scalar divide
  diff /= (np.cov(value_first, aweights=aweight_first, fweights=fweight_first) + np.cov(value_last, aweights=aweight_last, fweights=fweight_last))**0.5
/global/common/software/desi/users/adematti/perlmutter/cosmodesiconda/20240118-1.0.0/conda/lib/python3.10/site-packages/scipy/stats/_stats_py.py:1070: RuntimeWarning: invalid value encountered in subtract
  a_zero_mean = a - mean


[007890.29] [0/1] 12-13 07:30  MCMCSampler               INFO     499 steps, acceptance rate 0.433.
[007903.63] [0/1] 12-13 07:30  Chain                     INFO     Saving ../_tests/chain_desi_lrg_b1_omega_cdm_0.6-0.8.npy.
[007903.63] [0/1] 12-13 07:30  MCMCSampler               INFO     Diagnostics:
[007903.64] [0/1] 12-13 07:30  Diagnostics               INFO     - max eigen Gelman-Rubin - 1 is 0.854; not < 0.01.
[007903.65] [0/1] 12-13 07:30  Diagnostics               INFO     - max diag Gelman-Rubin - 1 is 0.904.
[007903.69] [0/1] 12-13 07:30  Diagnostics               INFO     - max diag Gelman-Rubin - 1 at 1.0 sigmas is 1.24.
[007903.69] [0/1] 12-13 07:30  Diagnostics               INFO     - max Geweke is 1.82.
[007903.70] [0/1] 12-13 07:30  Diagnostics               INFO     - Geweke p-value is 0.328.
[007903.70] [0/1] 12-13 07:30  Diagnostics               INFO     - effective sample size = (450 iterations / integrated autocorrelation time) is 27.8.
[007903.70] [0/1] 12-13 07

In [24]:
from desilike.samples import Chain, plotting

zrange = ['0.4-0.6', '0.6-0.8', '0.8-1.1']
chains = []
for z in zrange:
    chain = Chain.load(f'_tests/chain_desi_lrg_b1_omega_cdm_{z}.npy').remove_burnin(0.5)
    print(chain.to_stats(tablefmt='pretty'))
    chains.append(chain)
g = plots.get_subplot_plotter(width_inch=5)
plotting.plot_triangle(chains, fn=f'Practices/figs/Desi_LRG_separate_bins.pdf',g=g, labels = ['z0.4-0.6', 'z0.6-0.8', 'z0.8-1.1'], settings={'smooth_scale_2D' : 0.5})

[001846.62] [0/1] 12-18 22:55  Chain                     INFO     Loading _tests/chain_desi_lrg_b1_omega_cdm_0.4-0.6.npy.
+-----------+--------+--------+--------+--------+-----------------+-----------------+
|           | argmax |  mean  | median |  std   | quantile:1sigma | interval:1sigma |
+-----------+--------+--------+--------+--------+-----------------+-----------------+
| omega_cdm | 0.1260 | 0.1270 | 0.1271 | 0.0058 | -0.0056/+0.0060 | -0.0046/+0.0065 |
|    b1     | 1.645  | 1.640  | 1.638  | 0.039  |  -0.038/+0.038  |  -0.044/+0.030  |
+-----------+--------+--------+--------+--------+-----------------+-----------------+
[001846.66] [0/1] 12-18 22:55  Chain                     INFO     Loading _tests/chain_desi_lrg_b1_omega_cdm_0.6-0.8.npy.
+-----------+--------+--------+--------+--------+-----------------+-----------------+
|           | argmax |  mean  | median |  std   | quantile:1sigma | interval:1sigma |
+-----------+--------+--------+--------+--------+-----------------+-

### DESI FS LRG merged

In [ ]:
from desilike.theories.galaxy_clustering import KaiserTracerPowerSpectrumMultipoles, DirectPowerSpectrumTemplate
from desilike.observables.galaxy_clustering import TracerPowerSpectrumMultipolesObservable
from desilike.likelihoods import ObservablesGaussianLikelihood, SumLikelihood
from desilike.observables import ObservableCovariance
from pathlib import Path
from desilike import setup_logging

wigglez_cosmo = cosmoprimo.Cosmology(
        omega_cdm=0.1153,
        omega_b=0.02238,
        H0=68.9,
        tau_reio = 0.083,
        n_s = 0.964,
        logA = 3.084, engine='class'
    )
forfit_dir = Path('/global/cfs/cdirs/desi//survey/catalogs/Y1/LSS/iron/LSScats/v1.5/unblinded/desipipe/forfit_2pt/')
zrange = [(0.4, 0.6), (0.6, 0.8), (0.8, 1.1)]
likelihoods = []
for z in zrange:
    # call in the data
    ref_fn = forfit_dir / f'forfit_power_syst-rotation-hod-photo_klim_0-0.02-0.20_2-0.02-0.20_4-0.02-0.20_LRG_GCcomb_z{z[0]}-{z[1]}.npy'
    covariance = ObservableCovariance.load(ref_fn).select(projs=0, select_projs=True)
    cov_mat = covariance.view()
    # P(k)
    observable_power = covariance.observables('power')
    data = observable_power.view()
    zeff = observable_power.attrs['zeff']
    kin = observable_power.attrs['kin']
    wmatrix = observable_power.attrs['wmatrix'][:36,:kin.shape[0]]
    kout = np.arange(0.02,0.2,0.005)
    wshotnoise = observable_power.attrs['wshotnoise'][:36]

    # create the likelihood
    print(f'Creating likelihood for z range {z[0]}-{z[1]} with zeff={zeff}')
    template = DirectPowerSpectrumTemplate(fiducial=wigglez_cosmo, z=zeff)
    theory = KaiserTracerPowerSpectrumMultipoles(template=template, k=kin)
    theory.params['b1'].update(namespace=f'{zeff:.1f}')
    observable = TracerPowerSpectrumMultipolesObservable(theory=theory, data=data, wmatrix=wmatrix,ells=(0,), ellsin=(0,),k=kout,kin=kin, wshotnoise=wshotnoise)
    likelihood = ObservablesGaussianLikelihood(observables=observable, covariance=cov_mat)
    likelihoods.append(likelihood)

total_likelihood = SumLikelihood(likelihoods=likelihoods)
for params in ['h', 'omega_b', 'logA', 'sn0']:
    total_likelihood.varied_params[params].update(fixed=True)
#likelihood.all_params['b1'].update(prior={'limits': [0., 10.], 'latex': 'b_1'})
from desilike.samplers import MCMCSampler

setup_logging()
sampler = MCMCSampler(total_likelihood, save_fn=f'../_tests/chain_desi_lrg_merged_b1_omega_cdm.npy')
sampler.run(check={'max_eigen_gr': 0.01}, max_iterations=120000)

[011220.84] [0/1] 12-13 08:25  ObservableCovariance      INFO     Loading /global/cfs/cdirs/desi/survey/catalogs/Y1/LSS/iron/LSScats/v1.5/unblinded/desipipe/forfit_2pt/forfit_power_syst-rotation-hod-photo_klim_0-0.02-0.20_2-0.02-0.20_4-0.02-0.20_LRG_GCcomb_z0.4-0.6.npy.
Creating likelihood for z range 0.4-0.6 with zeff=0.5096288678782911
[011221.06] [0/1] 12-13 08:25  ObservableCovariance      INFO     Loading /global/cfs/cdirs/desi/survey/catalogs/Y1/LSS/iron/LSScats/v1.5/unblinded/desipipe/forfit_2pt/forfit_power_syst-rotation-hod-photo_klim_0-0.02-0.20_2-0.02-0.20_4-0.02-0.20_LRG_GCcomb_z0.6-0.8.npy.
Creating likelihood for z range 0.6-0.8 with zeff=0.7057956472488681
[011221.31] [0/1] 12-13 08:25  ObservableCovariance      INFO     Loading /global/cfs/cdirs/desi/survey/catalogs/Y1/LSS/iron/LSScats/v1.5/unblinded/desipipe/forfit_2pt/forfit_power_syst-rotation-hod-photo_klim_0-0.02-0.20_2-0.02-0.20_4-0.02-0.20_LRG_GCcomb_z0.8-1.1.npy.
Creating likelihood for z range 0.8-1.1 with zeff

/pscratch/sd/j/john0712/install/desilike/desilike/base.py:447: UserWarning: Derived parameter loglikelihood of <desilike.likelihoods.base.ObservablesGaussianLikelihood object at 0x7fd90866ba00> is already derived in <desilike.likelihoods.base.ObservablesGaussianLikelihood object at 0x7fd908680520>.
  if self.mpicomm.rank == 0: warnings.warn(msg)
/pscratch/sd/j/john0712/install/desilike/desilike/base.py:447: UserWarning: Derived parameter logprior of <desilike.likelihoods.base.ObservablesGaussianLikelihood object at 0x7fd90866ba00> is already derived in <desilike.likelihoods.base.ObservablesGaussianLikelihood object at 0x7fd908680520>.
  if self.mpicomm.rank == 0: warnings.warn(msg)
/pscratch/sd/j/john0712/install/desilike/desilike/base.py:447: UserWarning: Derived parameter loglikelihood of <desilike.likelihoods.base.ObservablesGaussianLikelihood object at 0x7fd908624a90> is already derived in <desilike.likelihoods.base.ObservablesGaussianLikelihood object at 0x7fd90866ba00>.
  if self

[000000.08] [0/1] 12-13 08:25  MCMCSampler               INFO     Varied parameters: ['omega_cdm', '0.9.b1', '0.7.b1', '0.5.b1'].
[000030.62] [0/1] 12-13 08:26  BasePipeline              INFO     Found speeds:
[000030.62] [0/1] 12-13 08:26  BasePipeline              INFO     - <desilike.theories.primordial_cosmology.Cosmoprimo object at 0x7fd908680490>: 169.53 iterations / second - 0.006 s / iteration
[000030.62] [0/1] 12-13 08:26  BasePipeline              INFO     - <desilike.theories.galaxy_clustering.base.APEffect object at 0x7fd908668310>: 26.29 iterations / second - 0.038 s / iteration
[000030.62] [0/1] 12-13 08:26  BasePipeline              INFO     - <desilike.theories.galaxy_clustering.power_template.DirectPowerSpectrumTemplate object at 0x7fd9087ea860>: 1.13 iterations / second - 0.882 s / iteration
[000030.62] [0/1] 12-13 08:26  BasePipeline              INFO     - <desilike.theories.galaxy_clustering.full_shape.KaiserPowerSpectrumMultipoles object at 0x7fd908682ec0>: 407.58

/global/common/software/desi/users/adematti/perlmutter/cosmodesiconda/20240118-1.0.0/conda/lib/python3.10/site-packages/scipy/stats/_stats_py.py:1806: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


[000672.63] [0/1] 12-13 08:36  MCMCSampler               INFO     422 steps, acceptance rate 0.472.
[000684.35] [0/1] 12-13 08:37  Chain                     INFO     Saving ../_tests/chain_desi_lrg_merged_b1_omega_cdm.npy.
[000684.35] [0/1] 12-13 08:37  MCMCSampler               INFO     Diagnostics:
[000684.37] [0/1] 12-13 08:37  Diagnostics               INFO     - max eigen Gelman-Rubin - 1 is 3.68; not < 0.01.
[000684.39] [0/1] 12-13 08:37  Diagnostics               INFO     - max diag Gelman-Rubin - 1 is 3.75.
[000684.45] [0/1] 12-13 08:37  Diagnostics               INFO     - max diag Gelman-Rubin - 1 at 1.0 sigmas is 6.75.
[000684.46] [0/1] 12-13 08:37  Diagnostics               INFO     - max Geweke is 3.69e+14.
[000684.46] [0/1] 12-13 08:37  Diagnostics               INFO     - Geweke p-value is 1.51e-09.
[000684.46] [0/1] 12-13 08:37  Diagnostics               INFO     - effective sample size = (300 iterations / integrated autocorrelation time) is 14.8.
[000684.46] [0/1] 12-1

In [23]:
chain = Chain.load(f'_tests/chain_desi_lrg_merged_b1_omega_cdm.npy').remove_burnin(0.5)
print(chain.to_stats(tablefmt='pretty'))
g = plots.get_subplot_plotter(width_inch=7)
plotting.plot_triangle(chain, g=g, fn=f'Practices/figs/Desi_LRG_merged.pdf', settings={'smooth_scale_2D' : 0.5})

[001800.35] [0/1] 12-18 22:54  Chain                     INFO     Loading _tests/chain_desi_lrg_merged_b1_omega_cdm.npy.
+-----------+--------+--------+--------+--------+-----------------+-----------------+
|           | argmax |  mean  | median |  std   | quantile:1sigma | interval:1sigma |
+-----------+--------+--------+--------+--------+-----------------+-----------------+
| omega_cdm | 0.1244 | 0.1243 | 0.1241 | 0.0031 | -0.0029/+0.0030 | -0.0036/+0.0022 |
|  0.9.b1   | 1.947  | 1.947  | 1.948  | 0.027  |  -0.027/+0.026  |  -0.028/+0.025  |
|  0.5.b1   | 1.656  | 1.656  | 1.656  | 0.025  |  -0.025/+0.025  |  -0.025/+0.025  |
|  0.7.b1   | 1.812  | 1.813  | 1.813  | 0.026  |  -0.025/+0.025  |  -0.025/+0.026  |
+-----------+--------+--------+--------+--------+-----------------+-----------------+
Removed no burn in
[001801.95] [0/1] 12-18 22:54  Plotting                  INFO     Saving figure to Practices/figs/Desi_LRG_merged.pdf.


### DESI BGS

In [2]:
from desilike.theories.galaxy_clustering import KaiserTracerPowerSpectrumMultipoles, DirectPowerSpectrumTemplate
from desilike.observables.galaxy_clustering import TracerPowerSpectrumMultipolesObservable
from desilike.likelihoods import ObservablesGaussianLikelihood, SumLikelihood
from desilike.observables import ObservableCovariance
from pathlib import Path
from desilike import setup_logging

wigglez_cosmo = cosmoprimo.Cosmology(
        omega_cdm=0.1153,
        omega_b=0.02238,
        H0=68.9,
        tau_reio = 0.083,
        n_s = 0.964,
        logA = 3.084, engine='class'
    )
forfit_dir = Path('/global/cfs/cdirs/desi//survey/catalogs/Y1/LSS/iron/LSScats/v1.5/unblinded/desipipe/forfit_2pt/')


# call in the data
ref_fn = forfit_dir / f'forfit_power_syst-rotation-hod-photo_klim_0-0.02-0.20_2-0.02-0.20_4-0.02-0.20_BGS_BRIGHT-21.5_GCcomb_z0.1-0.4.npy'
covariance = ObservableCovariance.load(ref_fn).select(projs=0, select_projs=True)
cov_mat = covariance.view()
# P(k)
observable_power = covariance.observables('power')
data = observable_power.view()
zeff = observable_power.attrs['zeff']
kin = observable_power.attrs['kin']
wmatrix = observable_power.attrs['wmatrix'][:36,:kin.shape[0]]
kout = np.arange(0.02,0.2,0.005)
wshotnoise = observable_power.attrs['wshotnoise'][:36]

# create the likelihood
print(f'Creating likelihood for z range 0.1-0.4 with zeff={zeff}')
template = DirectPowerSpectrumTemplate(fiducial=wigglez_cosmo, z=zeff)
theory = KaiserTracerPowerSpectrumMultipoles(template=template, k=kin)
observable = TracerPowerSpectrumMultipolesObservable(theory=theory, data=data, wmatrix=wmatrix,ells=(0,), ellsin=(0,),k=kout,kin=kin, wshotnoise=wshotnoise)
likelihood = ObservablesGaussianLikelihood(observables=observable, covariance=cov_mat)


for params in ['h', 'omega_b', 'logA', 'sn0']:
    likelihood.varied_params[params].update(fixed=True)
#likelihood.all_params['b1'].update(prior={'limits': [0., 10.], 'latex': 'b_1'})
from desilike.samplers import MCMCSampler

setup_logging()
sampler = MCMCSampler(likelihood, save_fn=f'../_tests/chain_desi_bgs_bright_b1_omega_cdm_0.1-0.4.npy')
sampler.run(check={'max_eigen_gr': 0.01}, max_iterations=120000)

[000001.57] [0/1] 12-14 01:17  ObservableCovariance      INFO     Loading /global/cfs/cdirs/desi/survey/catalogs/Y1/LSS/iron/LSScats/v1.5/unblinded/desipipe/forfit_2pt/forfit_power_syst-rotation-hod-photo_klim_0-0.02-0.20_2-0.02-0.20_4-0.02-0.20_BGS_BRIGHT-21.5_GCcomb_z0.1-0.4.npy.
Creating likelihood for z range 0.1-0.4 with zeff=0.29536404346937617
[000000.04] [0/1] 12-14 01:18  MCMCSampler               INFO     Varied parameters: ['omega_cdm', 'b1'].
[000010.98] [0/1] 12-14 01:18  BasePipeline              INFO     Found speeds:
[000010.98] [0/1] 12-14 01:18  BasePipeline              INFO     - <desilike.theories.primordial_cosmology.Cosmoprimo object at 0x7f69ec447700>: 176.35 iterations / second - 0.006 s / iteration
[000010.98] [0/1] 12-14 01:18  BasePipeline              INFO     - <desilike.theories.galaxy_clustering.base.APEffect object at 0x7f69ec48eda0>: 23.92 iterations / second - 0.042 s / iteration
[000010.98] [0/1] 12-14 01:18  BasePipeline              INFO     - <des

/global/common/software/desi/users/adematti/perlmutter/cosmodesiconda/20240118-1.0.0/conda/lib/python3.10/site-packages/scipy/stats/_stats_py.py:1806: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=8
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


[000214.78] [0/1] 12-14 01:21  MCMCSampler               INFO     Updating proposal covariance.
[000246.11] [0/1] 12-14 01:22  MCMCSampler               INFO     479 steps, acceptance rate 0.472.
[000257.60] [0/1] 12-14 01:22  Chain                     INFO     Saving ../_tests/chain_desi_bgs_bright_b1_omega_cdm_0.1-0.4.npy.
[000257.60] [0/1] 12-14 01:22  MCMCSampler               INFO     Diagnostics:
[000257.62] [0/1] 12-14 01:22  Diagnostics               INFO     - max eigen Gelman-Rubin - 1 is 2.32; not < 0.01.
[000257.62] [0/1] 12-14 01:22  Diagnostics               INFO     - max diag Gelman-Rubin - 1 is 2.47.
[000257.65] [0/1] 12-14 01:22  Diagnostics               INFO     - max diag Gelman-Rubin - 1 at 1.0 sigmas is 3.36.
[000257.66] [0/1] 12-14 01:22  Diagnostics               INFO     - max Geweke is 11.7.
[000257.66] [0/1] 12-14 01:22  Diagnostics               INFO     - Geweke p-value is 0.0714.
[000257.66] [0/1] 12-14 01:22  Diagnostics               INFO     - effectiv

[Chain(shape=(4800,), params=ParameterCollection(['omega_cdm', 'b1', 'fweight', 'logposterior', 'h', 'omega_b', 'logA', 'n_s', 'tau_reio', 'm_ncdm', 'N_eff', 'w0_fld', 'wa_fld', 'Omega_k', 'sigmapar', 'sigmaper', 'sn0', 'loglikelihood', 'logprior']))]

In [52]:
chain = Chain.load(f'_tests/chain_desi_bgs_bright_b1_omega_cdm_0.1-0.4.npy').remove_burnin(0.5)
print(chain.to_stats(tablefmt='pretty'))
plotting.plot_triangle(chain, fn=f'Practices/figs/Desi_BGS_bright.pdf', settings={'smooth_scale_2D' : 0.5})

[006680.84] [0/1] 12-14 22:34  Chain                     INFO     Loading _tests/chain_desi_bgs_bright_b1_omega_cdm_0.1-0.4.npy.
+-----------+--------+--------+--------+--------+-----------------+-----------------+
|           | argmax |  mean  | median |  std   | quantile:1sigma | interval:1sigma |
+-----------+--------+--------+--------+--------+-----------------+-----------------+
| omega_cdm | 0.1191 | 0.1199 | 0.1195 | 0.0090 | -0.0095/+0.0095 | -0.0135/+0.0036 |
|    b1     | 1.495  | 1.495  | 1.495  | 0.054  |  -0.055/+0.057  |  -0.052/+0.059  |
+-----------+--------+--------+--------+--------+-----------------+-----------------+
Removed no burn in
[006680.86] [0/1] 12-14 22:34  root                      WARNING  outlier fraction 0.004166666666666667 
[006681.14] [0/1] 12-14 22:34  Plotting                  INFO     Saving figure to Practices/figs/Desi_BGS_bright.pdf.


### DESI qso

In [3]:
from desilike.theories.galaxy_clustering import KaiserTracerPowerSpectrumMultipoles, DirectPowerSpectrumTemplate
from desilike.observables.galaxy_clustering import TracerPowerSpectrumMultipolesObservable
from desilike.likelihoods import ObservablesGaussianLikelihood, SumLikelihood
from desilike.observables import ObservableCovariance
from pathlib import Path
from desilike import setup_logging

wigglez_cosmo = cosmoprimo.Cosmology(
        omega_cdm=0.1153,
        omega_b=0.02238,
        H0=68.9,
        tau_reio = 0.083,
        n_s = 0.964,
        logA = 3.084, engine='class'
    )
forfit_dir = Path('/global/cfs/cdirs/desi//survey/catalogs/Y1/LSS/iron/LSScats/v1.5/unblinded/desipipe/forfit_2pt/')


# call in the data
ref_fn = forfit_dir / f'forfit_power_syst-rotation-hod-photo_klim_0-0.02-0.20_2-0.02-0.20_4-0.02-0.20_QSO_GCcomb_z0.8-2.1.npy'
covariance = ObservableCovariance.load(ref_fn).select(projs=0, select_projs=True)
cov_mat = covariance.view()
# P(k)
observable_power = covariance.observables('power')
data = observable_power.view()
zeff = observable_power.attrs['zeff']
kin = observable_power.attrs['kin']
wmatrix = observable_power.attrs['wmatrix'][:36,:kin.shape[0]]
kout = np.arange(0.02,0.2,0.005)
wshotnoise = observable_power.attrs['wshotnoise'][:36]

# create the likelihood
print(f'Creating likelihood for z range 0.8-2.1 with zeff={zeff}')
template = DirectPowerSpectrumTemplate(fiducial=wigglez_cosmo, z=zeff)
theory = KaiserTracerPowerSpectrumMultipoles(template=template, k=kin)
observable = TracerPowerSpectrumMultipolesObservable(theory=theory, data=data, wmatrix=wmatrix,ells=(0,), ellsin=(0,),k=kout,kin=kin, wshotnoise=wshotnoise)
likelihood = ObservablesGaussianLikelihood(observables=observable, covariance=cov_mat)


for params in ['h', 'omega_b', 'logA', 'sn0']:
    likelihood.varied_params[params].update(fixed=True)
#likelihood.all_params['b1'].update(prior={'limits': [0., 10.], 'latex': 'b_1'})
from desilike.samplers import MCMCSampler

setup_logging()
sampler = MCMCSampler(likelihood, save_fn=f'../_tests/chain_desi_qso_b1_omega_cdm_0.8-2.1.npy')
sampler.run(check={'max_eigen_gr': 0.01}, max_iterations=120000)

[001277.11] [0/1] 12-14 01:39  ObservableCovariance      INFO     Loading /global/cfs/cdirs/desi/survey/catalogs/Y1/LSS/iron/LSScats/v1.5/unblinded/desipipe/forfit_2pt/forfit_power_syst-rotation-hod-photo_klim_0-0.02-0.20_2-0.02-0.20_4-0.02-0.20_QSO_GCcomb_z0.8-2.1.npy.
Creating likelihood for z range 0.8-2.1 with zeff=1.4905017757527006
[000000.04] [0/1] 12-14 01:39  MCMCSampler               INFO     Varied parameters: ['omega_cdm', 'b1'].
[000010.37] [0/1] 12-14 01:39  BasePipeline              INFO     Found speeds:
[000010.37] [0/1] 12-14 01:39  BasePipeline              INFO     - <desilike.theories.primordial_cosmology.Cosmoprimo object at 0x7f69ec15aef0>: 170.69 iterations / second - 0.006 s / iteration
[000010.38] [0/1] 12-14 01:39  BasePipeline              INFO     - <desilike.theories.galaxy_clustering.base.APEffect object at 0x7f6b77af22f0>: 23.96 iterations / second - 0.042 s / iteration
[000010.38] [0/1] 12-14 01:39  BasePipeline              INFO     - <desilike.theorie

[Chain(shape=(12600,), params=ParameterCollection(['omega_cdm', 'b1', 'fweight', 'logposterior', 'h', 'omega_b', 'logA', 'n_s', 'tau_reio', 'm_ncdm', 'N_eff', 'w0_fld', 'wa_fld', 'Omega_k', 'sigmapar', 'sigmaper', 'sn0', 'loglikelihood', 'logprior']))]

In [53]:
chain = Chain.load(f'_tests/chain_desi_qso_b1_omega_cdm_0.8-2.1.npy').remove_burnin(0.5)
print(chain.to_stats(tablefmt='pretty'))
plotting.plot_triangle(chain, fn=f'Practices/figs/Desi_QSO.pdf', settings={'smooth_scale_2D' : 0.5})

[006732.59] [0/1] 12-14 22:35  Chain                     INFO     Loading _tests/chain_desi_qso_b1_omega_cdm_0.8-2.1.npy.
+-----------+--------+--------+--------+--------+-----------------+-----------------+
|           | argmax |  mean  | median |  std   | quantile:1sigma | interval:1sigma |
+-----------+--------+--------+--------+--------+-----------------+-----------------+
| omega_cdm | 0.1415 | 0.1413 | 0.1411 | 0.0073 | -0.0078/+0.0074 | -0.0084/+0.0061 |
|    b1     | 1.945  | 1.948  | 1.948  | 0.048  |  -0.047/+0.048  |  -0.049/+0.045  |
+-----------+--------+--------+--------+--------+-----------------+-----------------+
Removed no burn in
[006732.92] [0/1] 12-14 22:35  Plotting                  INFO     Saving figure to Practices/figs/Desi_QSO.pdf.


### DESI ELG

In [4]:
from desilike.theories.galaxy_clustering import KaiserTracerPowerSpectrumMultipoles, DirectPowerSpectrumTemplate
from desilike.observables.galaxy_clustering import TracerPowerSpectrumMultipolesObservable
from desilike.likelihoods import ObservablesGaussianLikelihood, SumLikelihood
from desilike.observables import ObservableCovariance
from pathlib import Path
from desilike import setup_logging

wigglez_cosmo = cosmoprimo.Cosmology(
        omega_cdm=0.1153,
        omega_b=0.02238,
        H0=68.9,
        tau_reio = 0.083,
        n_s = 0.964,
        logA = 3.084, engine='class'
    )
forfit_dir = Path('/global/cfs/cdirs/desi//survey/catalogs/Y1/LSS/iron/LSScats/v1.5/unblinded/desipipe/forfit_2pt/')
zrange = [(0.8, 1.1), (1.1,1.6)]
likelihoods = []
for z in zrange:
    # call in the data
    ref_fn = forfit_dir / f'forfit_power_syst-rotation-hod-photo_klim_0-0.02-0.20_2-0.02-0.20_4-0.02-0.20_ELG_LOPnotqso_GCcomb_z{z[0]}-{z[1]}.npy'
    covariance = ObservableCovariance.load(ref_fn).select(projs=0, select_projs=True)
    cov_mat = covariance.view()
    # P(k)
    observable_power = covariance.observables('power')
    data = observable_power.view()
    zeff = observable_power.attrs['zeff']
    kin = observable_power.attrs['kin']
    wmatrix = observable_power.attrs['wmatrix'][:36,:kin.shape[0]]
    kout = np.arange(0.02,0.2,0.005)
    wshotnoise = observable_power.attrs['wshotnoise'][:36]

    # create the likelihood
    print(f'Creating likelihood for z range {z[0]}-{z[1]} with zeff={zeff}')
    template = DirectPowerSpectrumTemplate(fiducial=wigglez_cosmo, z=zeff)
    theory = KaiserTracerPowerSpectrumMultipoles(template=template, k=kin)
    theory.params['b1'].update(namespace=f'{zeff:.1f}')
    observable = TracerPowerSpectrumMultipolesObservable(theory=theory, data=data, wmatrix=wmatrix,ells=(0,), ellsin=(0,),k=kout,kin=kin, wshotnoise=wshotnoise)
    likelihood = ObservablesGaussianLikelihood(observables=observable, covariance=cov_mat)
    likelihoods.append(likelihood)

total_likelihood = SumLikelihood(likelihoods=likelihoods)
for params in ['h', 'omega_b', 'logA', 'sn0']:
    total_likelihood.varied_params[params].update(fixed=True)
#likelihood.all_params['b1'].update(prior={'limits': [0., 10.], 'latex': 'b_1'})
from desilike.samplers import MCMCSampler

setup_logging()
sampler = MCMCSampler(total_likelihood, save_fn=f'../_tests/chain_desi_elg_merged_b1_omega_cdm.npy')
sampler.run(check={'max_eigen_gr': 0.01}, max_iterations=120000)

[002199.19] [0/1] 12-14 02:15  ObservableCovariance      INFO     Loading /global/cfs/cdirs/desi/survey/catalogs/Y1/LSS/iron/LSScats/v1.5/unblinded/desipipe/forfit_2pt/forfit_power_syst-rotation-hod-photo_klim_0-0.02-0.20_2-0.02-0.20_4-0.02-0.20_ELG_LOPnotqso_GCcomb_z0.8-1.1.npy.
Creating likelihood for z range 0.8-1.1 with zeff=0.955156501837269
[002199.52] [0/1] 12-14 02:15  ObservableCovariance      INFO     Loading /global/cfs/cdirs/desi/survey/catalogs/Y1/LSS/iron/LSScats/v1.5/unblinded/desipipe/forfit_2pt/forfit_power_syst-rotation-hod-photo_klim_0-0.02-0.20_2-0.02-0.20_4-0.02-0.20_ELG_LOPnotqso_GCcomb_z1.1-1.6.npy.
Creating likelihood for z range 1.1-1.6 with zeff=1.3170658832980264


/pscratch/sd/j/john0712/install/desilike/desilike/base.py:447: UserWarning: Derived parameter loglikelihood of <desilike.likelihoods.base.ObservablesGaussianLikelihood object at 0x7f69cc54e3e0> is already derived in <desilike.likelihoods.base.ObservablesGaussianLikelihood object at 0x7f69cc4d1840>.
  if self.mpicomm.rank == 0: warnings.warn(msg)
/pscratch/sd/j/john0712/install/desilike/desilike/base.py:447: UserWarning: Derived parameter logprior of <desilike.likelihoods.base.ObservablesGaussianLikelihood object at 0x7f69cc54e3e0> is already derived in <desilike.likelihoods.base.ObservablesGaussianLikelihood object at 0x7f69cc4d1840>.
  if self.mpicomm.rank == 0: warnings.warn(msg)
/pscratch/sd/j/john0712/install/desilike/desilike/base.py:447: UserWarning: Derived parameter loglikelihood of <desilike.likelihoods.base.SumLikelihood object at 0x7f69cc4d3310> is already derived in <desilike.likelihoods.base.ObservablesGaussianLikelihood object at 0x7f69cc54e3e0>.
  if self.mpicomm.rank ==

[000000.05] [0/1] 12-14 02:15  MCMCSampler               INFO     Varied parameters: ['omega_cdm', '1.3.b1', '1.0.b1'].
[000020.55] [0/1] 12-14 02:16  BasePipeline              INFO     Found speeds:
[000020.55] [0/1] 12-14 02:16  BasePipeline              INFO     - <desilike.theories.primordial_cosmology.Cosmoprimo object at 0x7f69cc54c580>: 164.96 iterations / second - 0.006 s / iteration
[000020.55] [0/1] 12-14 02:16  BasePipeline              INFO     - <desilike.theories.galaxy_clustering.base.APEffect object at 0x7f69cc37a320>: 23.50 iterations / second - 0.043 s / iteration
[000020.55] [0/1] 12-14 02:16  BasePipeline              INFO     - <desilike.theories.galaxy_clustering.power_template.DirectPowerSpectrumTemplate object at 0x7f69cc4d5a20>: 1.14 iterations / second - 0.877 s / iteration
[000020.55] [0/1] 12-14 02:16  BasePipeline              INFO     - <desilike.theories.galaxy_clustering.full_shape.KaiserPowerSpectrumMultipoles object at 0x7f69cc498a00>: 492.70 iteration

/global/common/software/desi/users/adematti/perlmutter/cosmodesiconda/20240118-1.0.0/conda/lib/python3.10/site-packages/scipy/stats/_stats_py.py:1806: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=12
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


[000488.63] [0/1] 12-14 02:24  MCMCSampler               INFO     Updating proposal covariance.
[000521.31] [0/1] 12-14 02:24  MCMCSampler               INFO     482 steps, acceptance rate 0.448.
[000534.57] [0/1] 12-14 02:24  Chain                     INFO     Saving ../_tests/chain_desi_elg_merged_b1_omega_cdm.npy.
[000534.57] [0/1] 12-14 02:24  MCMCSampler               INFO     Diagnostics:
[000534.59] [0/1] 12-14 02:24  Diagnostics               INFO     - max eigen Gelman-Rubin - 1 is 1.31; not < 0.01.
[000534.60] [0/1] 12-14 02:24  Diagnostics               INFO     - max diag Gelman-Rubin - 1 is 0.605.
[000534.65] [0/1] 12-14 02:24  Diagnostics               INFO     - max diag Gelman-Rubin - 1 at 1.0 sigmas is 2.43.
[000534.66] [0/1] 12-14 02:24  Diagnostics               INFO     - max Geweke is 1.6e+13.
[000534.66] [0/1] 12-14 02:24  Diagnostics               INFO     - Geweke p-value is 4.94e-08.
[000534.66] [0/1] 12-14 02:24  Diagnostics               INFO     - effective 

[Chain(shape=(24000,), params=ParameterCollection(['omega_cdm', '1.3.b1', '1.0.b1', 'fweight', 'logposterior', 'h', 'omega_b', 'logA', 'n_s', 'tau_reio', 'm_ncdm', 'N_eff', 'w0_fld', 'wa_fld', 'Omega_k', 'sigmapar', 'sigmaper', 'sn0', 'loglikelihood', 'logprior']))]

In [55]:
chain = Chain.load(f'_tests/chain_desi_elg_merged_b1_omega_cdm.npy').remove_burnin(0.5)
print(chain.to_stats(tablefmt='pretty'))
plotting.plot_triangle(chain, params = ['omega_cdm', '1.0.b1', '1.3.b1'],fn=f'Practices/figs/Desi_ELG_merged.pdf', settings={'smooth_scale_2D' : 0.5})

[006939.66] [0/1] 12-14 22:38  Chain                     INFO     Loading _tests/chain_desi_elg_merged_b1_omega_cdm.npy.
+-----------+--------+--------+--------+--------+-----------------+-----------------+
|           | argmax |  mean  | median |  std   | quantile:1sigma | interval:1sigma |
+-----------+--------+--------+--------+--------+-----------------+-----------------+
| omega_cdm | 0.1347 | 0.1349 | 0.1354 | 0.0062 | -0.0061/+0.0064 | -0.0048/+0.0073 |
|  1.3.b1   | 1.170  | 1.170  | 1.168  | 0.033  |  -0.033/+0.033  |  -0.038/+0.028  |
|  1.0.b1   | 0.904  | 0.904  | 0.903  | 0.026  |  -0.026/+0.026  |  -0.028/+0.022  |
+-----------+--------+--------+--------+--------+-----------------+-----------------+
Removed no burn in
[006940.44] [0/1] 12-14 22:38  Plotting                  INFO     Saving figure to Practices/figs/Desi_ELG_merged.pdf.


### DESI merged all combined

In [ ]:
from desilike.theories.galaxy_clustering import KaiserTracerPowerSpectrumMultipoles, DirectPowerSpectrumTemplate
from desilike.observables.galaxy_clustering import TracerPowerSpectrumMultipolesObservable
from desilike.likelihoods import ObservablesGaussianLikelihood, SumLikelihood
from desilike.observables import ObservableCovariance
from pathlib import Path
from desilike import setup_logging

wigglez_cosmo = cosmoprimo.Cosmology(
        omega_cdm=0.1153,
        omega_b=0.02238,
        H0=68.9,
        tau_reio = 0.083,
        n_s = 0.964,
        logA = 3.084, engine='class'
    )
forfit_dir = Path('/global/cfs/cdirs/desi//survey/catalogs/Y1/LSS/iron/LSScats/v1.5/unblinded/desipipe/forfit_2pt/')
likelihoods = []
# BGS
# call in the data
ref_fn = forfit_dir / f'forfit_power_syst-rotation-hod-photo_klim_0-0.02-0.20_2-0.02-0.20_4-0.02-0.20_BGS_BRIGHT-21.5_GCcomb_z0.1-0.4.npy'
covariance = ObservableCovariance.load(ref_fn).select(projs=0, select_projs=True)
cov_mat = covariance.view()
# P(k)
observable_power = covariance.observables('power')
data = observable_power.view()
zeff = observable_power.attrs['zeff']
kin = observable_power.attrs['kin']
wmatrix = observable_power.attrs['wmatrix'][:36,:kin.shape[0]]
kout = np.arange(0.02,0.2,0.005)
wshotnoise = observable_power.attrs['wshotnoise'][:36]

# create the likelihood
print(f'Creating likelihood for z range 0.1-0.4 with zeff={zeff}')
template = DirectPowerSpectrumTemplate(fiducial=wigglez_cosmo, z=zeff)
theory = KaiserTracerPowerSpectrumMultipoles(template=template, k=kin)
theory.params['b1'].update(namespace=f'bgs{zeff:.1f}')
observable = TracerPowerSpectrumMultipolesObservable(theory=theory, data=data, wmatrix=wmatrix,ells=(0,), ellsin=(0,),k=kout,kin=kin, wshotnoise=wshotnoise)
likelihood = ObservablesGaussianLikelihood(observables=observable, covariance=cov_mat)
likelihoods.append(likelihood)
# LRG
zrange = [(0.4, 0.6), (0.6, 0.8), (0.8, 1.1)]
for z in zrange:
    # call in the data
    ref_fn = forfit_dir / f'forfit_power_syst-rotation-hod-photo_klim_0-0.02-0.20_2-0.02-0.20_4-0.02-0.20_LRG_GCcomb_z{z[0]}-{z[1]}.npy'
    covariance = ObservableCovariance.load(ref_fn).select(projs=0, select_projs=True)
    cov_mat = covariance.view()
    # P(k)
    observable_power = covariance.observables('power')
    data = observable_power.view()
    zeff = observable_power.attrs['zeff']
    kin = observable_power.attrs['kin']
    wmatrix = observable_power.attrs['wmatrix'][:36,:kin.shape[0]]
    kout = np.arange(0.02,0.2,0.005)
    wshotnoise = observable_power.attrs['wshotnoise'][:36]

    # create the likelihood
    print(f'Creating likelihood for z range {z[0]}-{z[1]} with zeff={zeff}')
    template = DirectPowerSpectrumTemplate(fiducial=wigglez_cosmo, z=zeff)
    theory = KaiserTracerPowerSpectrumMultipoles(template=template, k=kin)
    theory.params['b1'].update(namespace=f'lrg{zeff:.1f}')
    observable = TracerPowerSpectrumMultipolesObservable(theory=theory, data=data, wmatrix=wmatrix,ells=(0,), ellsin=(0,),k=kout,kin=kin, wshotnoise=wshotnoise)
    likelihood = ObservablesGaussianLikelihood(observables=observable, covariance=cov_mat)
    likelihoods.append(likelihood)
# ELG
zrange = [(0.8, 1.1), (1.1,1.6)]
for z in zrange:
    # call in the data
    ref_fn = forfit_dir / f'forfit_power_syst-rotation-hod-photo_klim_0-0.02-0.20_2-0.02-0.20_4-0.02-0.20_ELG_LOPnotqso_GCcomb_z{z[0]}-{z[1]}.npy'
    covariance = ObservableCovariance.load(ref_fn).select(projs=0, select_projs=True)
    cov_mat = covariance.view()
    # P(k)
    observable_power = covariance.observables('power')
    data = observable_power.view()
    zeff = observable_power.attrs['zeff']
    kin = observable_power.attrs['kin']
    wmatrix = observable_power.attrs['wmatrix'][:36,:kin.shape[0]]
    kout = np.arange(0.02,0.2,0.005)
    wshotnoise = observable_power.attrs['wshotnoise'][:36]

    # create the likelihood
    print(f'Creating likelihood for z range {z[0]}-{z[1]} with zeff={zeff}')
    template = DirectPowerSpectrumTemplate(fiducial=wigglez_cosmo, z=zeff)
    theory = KaiserTracerPowerSpectrumMultipoles(template=template, k=kin)
    theory.params['b1'].update(namespace=f'elg{zeff:.1f}')
    observable = TracerPowerSpectrumMultipolesObservable(theory=theory, data=data, wmatrix=wmatrix,ells=(0,), ellsin=(0,),k=kout,kin=kin, wshotnoise=wshotnoise)
    likelihood = ObservablesGaussianLikelihood(observables=observable, covariance=cov_mat)
    likelihoods.append(likelihood)
# QSO
# call in the data
ref_fn = forfit_dir / f'forfit_power_syst-rotation-hod-photo_klim_0-0.02-0.20_2-0.02-0.20_4-0.02-0.20_QSO_GCcomb_z0.8-2.1.npy'
covariance = ObservableCovariance.load(ref_fn).select(projs=0, select_projs=True)
cov_mat = covariance.view()
# P(k)
observable_power = covariance.observables('power')
data = observable_power.view()
zeff = observable_power.attrs['zeff']
kin = observable_power.attrs['kin']
wmatrix = observable_power.attrs['wmatrix'][:36,:kin.shape[0]]
kout = np.arange(0.02,0.2,0.005)
wshotnoise = observable_power.attrs['wshotnoise'][:36]

# create the likelihood
print(f'Creating likelihood for z range 0.8-2.1 with zeff={zeff}')
template = DirectPowerSpectrumTemplate(fiducial=wigglez_cosmo, z=zeff)
theory = KaiserTracerPowerSpectrumMultipoles(template=template, k=kin)
theory.params['b1'].update(namespace=f'qso{zeff:.1f}')
observable = TracerPowerSpectrumMultipolesObservable(theory=theory, data=data, wmatrix=wmatrix,ells=(0,), ellsin=(0,),k=kout,kin=kin, wshotnoise=wshotnoise)
likelihood = ObservablesGaussianLikelihood(observables=observable, covariance=cov_mat)
likelihoods.append(likelihood)



total_likelihood = SumLikelihood(likelihoods=likelihoods)
for params in ['h', 'omega_b', 'logA', 'sn0']:
    total_likelihood.varied_params[params].update(fixed=True)
#likelihood.all_params['b1'].update(prior={'limits': [0., 10.], 'latex': 'b_1'})
from desilike.samplers import MCMCSampler

setup_logging()
sampler = MCMCSampler(total_likelihood, save_fn=f'../_tests/chain_desi_tot_merged_b1_omega_cdm.npy')
sampler.run(check={'max_eigen_gr': 0.01}, max_iterations=160000)

[004503.33] [0/1] 12-14 03:31  ObservableCovariance      INFO     Loading /global/cfs/cdirs/desi/survey/catalogs/Y1/LSS/iron/LSScats/v1.5/unblinded/desipipe/forfit_2pt/forfit_power_syst-rotation-hod-photo_klim_0-0.02-0.20_2-0.02-0.20_4-0.02-0.20_BGS_BRIGHT-21.5_GCcomb_z0.1-0.4.npy.
Creating likelihood for z range 0.1-0.4 with zeff=0.29536404346937617
[004503.57] [0/1] 12-14 03:31  ObservableCovariance      INFO     Loading /global/cfs/cdirs/desi/survey/catalogs/Y1/LSS/iron/LSScats/v1.5/unblinded/desipipe/forfit_2pt/forfit_power_syst-rotation-hod-photo_klim_0-0.02-0.20_2-0.02-0.20_4-0.02-0.20_LRG_GCcomb_z0.4-0.6.npy.
Creating likelihood for z range 0.4-0.6 with zeff=0.5096288678782911
[004503.90] [0/1] 12-14 03:31  ObservableCovariance      INFO     Loading /global/cfs/cdirs/desi/survey/catalogs/Y1/LSS/iron/LSScats/v1.5/unblinded/desipipe/forfit_2pt/forfit_power_syst-rotation-hod-photo_klim_0-0.02-0.20_2-0.02-0.20_4-0.02-0.20_LRG_GCcomb_z0.6-0.8.npy.
Creating likelihood for z range 0.6-

/pscratch/sd/j/john0712/install/desilike/desilike/base.py:447: UserWarning: Derived parameter loglikelihood of <desilike.likelihoods.base.ObservablesGaussianLikelihood object at 0x7f69cc2a9b40> is already derived in <desilike.likelihoods.base.ObservablesGaussianLikelihood object at 0x7f6b7792df30>.
  if self.mpicomm.rank == 0: warnings.warn(msg)
/pscratch/sd/j/john0712/install/desilike/desilike/base.py:447: UserWarning: Derived parameter logprior of <desilike.likelihoods.base.ObservablesGaussianLikelihood object at 0x7f69cc2a9b40> is already derived in <desilike.likelihoods.base.ObservablesGaussianLikelihood object at 0x7f6b7792df30>.
  if self.mpicomm.rank == 0: warnings.warn(msg)
/pscratch/sd/j/john0712/install/desilike/desilike/base.py:447: UserWarning: Derived parameter loglikelihood of <desilike.likelihoods.base.ObservablesGaussianLikelihood object at 0x7f69cc22df00> is already derived in <desilike.likelihoods.base.ObservablesGaussianLikelihood object at 0x7f69cc2a9b40>.
  if self

[000000.21] [0/1] 12-14 03:31  MCMCSampler               INFO     Varied parameters: ['omega_cdm', 'qso1.5.b1', 'elg1.3.b1', 'elg1.0.b1', 'lrg0.9.b1', 'lrg0.7.b1', 'lrg0.5.b1', 'bgs0.3.b1'].
[000072.22] [0/1] 12-14 03:32  BasePipeline              INFO     Found speeds:
[000072.23] [0/1] 12-14 03:32  BasePipeline              INFO     - <desilike.theories.primordial_cosmology.Cosmoprimo object at 0x7f69cc10bf40>: 177.45 iterations / second - 0.006 s / iteration
[000072.23] [0/1] 12-14 03:32  BasePipeline              INFO     - <desilike.theories.galaxy_clustering.base.APEffect object at 0x7f69cc10bc40>: 30.34 iterations / second - 0.033 s / iteration
[000072.23] [0/1] 12-14 03:32  BasePipeline              INFO     - <desilike.theories.galaxy_clustering.power_template.DirectPowerSpectrumTemplate object at 0x7f69ec1c33d0>: 1.10 iterations / second - 0.913 s / iteration
[000072.23] [0/1] 12-14 03:32  BasePipeline              INFO     - <desilike.theories.galaxy_clustering.full_shape.Ka

/pscratch/sd/j/john0712/install/desilike/desilike/samples/diagnostics.py:339: RuntimeWarning: divide by zero encountered in scalar divide
  diff /= (np.cov(value_first, aweights=aweight_first, fweights=fweight_first) + np.cov(value_last, aweights=aweight_last, fweights=fweight_last))**0.5
/global/common/software/desi/users/adematti/perlmutter/cosmodesiconda/20240118-1.0.0/conda/lib/python3.10/site-packages/scipy/stats/_stats_py.py:1070: RuntimeWarning: invalid value encountered in subtract
  a_zero_mean = a - mean


[000649.35] [0/1] 12-14 03:41  MCMCSampler               INFO     523 steps, acceptance rate 0.440.
[000662.30] [0/1] 12-14 03:42  Chain                     INFO     Saving ../_tests/chain_desi_tot_merged_b1_omega_cdm.npy.
[000662.31] [0/1] 12-14 03:42  MCMCSampler               INFO     Diagnostics:
[000662.34] [0/1] 12-14 03:42  Diagnostics               INFO     - max eigen Gelman-Rubin - 1 is 11.6; not < 0.01.
[000662.37] [0/1] 12-14 03:42  Diagnostics               INFO     - max diag Gelman-Rubin - 1 is 8.55.
[000662.49] [0/1] 12-14 03:42  Diagnostics               INFO     - max diag Gelman-Rubin - 1 at 1.0 sigmas is 10.6.
[000662.51] [0/1] 12-14 03:42  Diagnostics               INFO     - max Geweke is nan.
[000662.51] [0/1] 12-14 03:42  Diagnostics               INFO     - Geweke p-value is nan.
[000662.52] [0/1] 12-14 03:42  Diagnostics               INFO     - effective sample size = (300 iterations / integrated autocorrelation time) is 8.16.
[000662.52] [0/1] 12-14 03:42  D

/pscratch/sd/j/john0712/install/desilike/desilike/samples/diagnostics.py:339: RuntimeWarning: invalid value encountered in scalar divide
  diff /= (np.cov(value_first, aweights=aweight_first, fweights=fweight_first) + np.cov(value_last, aweights=aweight_last, fweights=fweight_last))**0.5


[000692.46] [0/1] 12-14 03:42  Chain                     INFO     Saving ../_tests/chain_desi_tot_merged_b1_omega_cdm.npy.
[000692.46] [0/1] 12-14 03:42  MCMCSampler               INFO     Diagnostics:
[000692.49] [0/1] 12-14 03:42  Diagnostics               INFO     - max eigen Gelman-Rubin - 1 is 52.9; not < 0.01.
[000692.51] [0/1] 12-14 03:42  Diagnostics               INFO     - max diag Gelman-Rubin - 1 is 25.7.
[000692.61] [0/1] 12-14 03:42  Diagnostics               INFO     - max diag Gelman-Rubin - 1 at 1.0 sigmas is 29.4.
[000692.63] [0/1] 12-14 03:42  Diagnostics               INFO     - max Geweke is nan.
[000692.63] [0/1] 12-14 03:42  Diagnostics               INFO     - Geweke p-value is nan.
[000692.63] [0/1] 12-14 03:42  Diagnostics               INFO     - effective sample size = (450 iterations / integrated autocorrelation time) is 8.14.
[000692.63] [0/1] 12-14 03:42  Diagnostics               INFO     - max variation of integrated autocorrelation time is 1.03.
[00069

In [58]:
chain = Chain.load(f'_tests/chain_desi_tot_merged_b1_omega_cdm.npy').remove_burnin(0.5)
print(chain.to_stats(tablefmt='pretty'))
plotting.plot_triangle(chain,params=['omega_cdm', 'bgs0.3.b1', 'lrg0.5.b1', 'lrg0.7.b1', 'lrg0.9.b1', 'elg1.0.b1', 'elg1.3.b1', 'qso1.5.b1'], fn=f'Practices/figs/Desi_tot_merged.pdf', settings={'smooth_scale_2D' : 0.5})

[007075.37] [0/1] 12-14 22:40  Chain                     INFO     Loading _tests/chain_desi_tot_merged_b1_omega_cdm.npy.
+-----------+--------+--------+--------+--------+-----------------+-----------------+
|           | argmax |  mean  | median |  std   | quantile:1sigma | interval:1sigma |
+-----------+--------+--------+--------+--------+-----------------+-----------------+
| omega_cdm | 0.1282 | 0.1281 | 0.1282 | 0.0023 | -0.0026/+0.0023 | -0.0022/+0.0024 |
| lrg0.9.b1 | 1.921  | 1.920  | 1.920  | 0.022  |  -0.022/+0.022  |  -0.023/+0.021  |
| elg1.3.b1 | 1.199  | 1.202  | 1.201  | 0.019  |  -0.019/+0.020  |  -0.021/+0.018  |
| lrg0.5.b1 | 1.634  | 1.633  | 1.633  | 0.020  |  -0.021/+0.021  |  -0.020/+0.022  |
| elg1.0.b1 | 0.923  | 0.929  | 0.929  | 0.016  |  -0.016/+0.016  |  -0.017/+0.016  |
| qso1.5.b1 | 2.028  | 2.029  | 2.029  | 0.028  |  -0.029/+0.028  |  -0.028/+0.029  |
| lrg0.7.b1 | 1.784  | 1.788  | 1.788  | 0.021  |  -0.021/+0.021  |  -0.021/+0.021  |
| bgs0.3.b1 | 1.447

### Further analysis

In [2]:
from desilike import utils
import numpy as np
from scipy import special

mu, wmu = utils.weights_mu(mu=8, method='trapz')
ell = 0
np.array([wmu * (2 * ell + 1) * special.legendre(ell)(mu)])

array([[0.07142857, 0.14285714, 0.14285714, 0.14285714, 0.14285714,
        0.14285714, 0.14285714, 0.07142857]])

In [8]:
mu

array([0.        , 0.14285714, 0.28571429, 0.42857143, 0.57142857,
       0.71428571, 0.85714286, 1.        ])

In [9]:
wmu

Array([0.07142857, 0.14285714, 0.14285714, 0.14285714, 0.14285714,
       0.14285714, 0.14285714, 0.07142857], dtype=float64)